# Classification with Iris Dataset

This notebook demonstrates how to:

1. Define the model input and output schema
1. Load in data from a text file to an IDataView
1. Set up the training pipeline with data transforms
1. Choose an algorithm and append it to the pipeline
1. Train the model
1. Evaluate the model
1. Consume the model

## Install the necessary NuGet packages for training ML.NET model and plotting:

In [1]:

/* Notebook files contain both code snippets and rich text elements.
Use the "run" button in the left margin to execute each code snippet and explore ML.NET. */

// using nightly-build
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json"
#r "nuget: Microsoft.ML.AutoML, 0.21.0-preview.23504.1"
#r "nuget: Microsoft.Data.Analysis, 0.21.0-preview.23504.1"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.0-preview.23504.1 Microsoft.ML.AutoML, 0.21.0-preview.23504.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\microsoft.data.analysis\0.21.0-preview.23504.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\plotly.net.interactive\3.0.2\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\microsoft.ml.automl\0.21.0-preview.23504.1\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

In [2]:

// Import common usings.
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.AutoML;

## Define the model input and output schemas:

In [3]:
// Define the model input schema (which columns you will be loading in for training)
public class ModelInput
{
    [ColumnName(@"Label"), LoadColumn(0)]
    public string Label { get; set; }
    
    [ColumnName(@"Sepal length"), LoadColumn(1)]
    public float Sepal_length { get; set; }
    
    [ColumnName(@"Sepal width"), LoadColumn(2)]
    public float Sepal_width { get; set; }
    
    [ColumnName(@"Petal length"), LoadColumn(3)]
    public float Petal_length { get; set; }
    
    [ColumnName(@"Petal width"), LoadColumn(4)]
    public float Petal_width { get; set; }
    
}


// Define the model output schema (what the model will return)
public class ModelOutput
{
    [ColumnName("PredictedLabel")]
    public string PredictedLabel { get; set;}

    public float[] Score { get; set;}
}



## Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [4]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

## Create MLContext and load training data:

In [5]:
// Create a new MLContext (the starting point for all ML.NET operations)
var mlContext = new MLContext();

// Define path to training data
string trainValidateDataPath =  EnsureDataSetDownloaded("iris-train.tsv");
string testDataPath = EnsureDataSetDownloaded("iris-test.tsv");

// Load data from a text file to an IDataView (a flexible, efficient way of describing tabular data)
IDataView trainValidateData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: trainValidateDataPath,
    hasHeader: true ,
    separatorChar: '\t',
    allowQuoting: true,
    allowSparse: false);

IDataView testData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: testDataPath,
    hasHeader: true ,
    separatorChar: '\t',
    allowQuoting: true,
    allowSparse: false);

// Display training data schema
display(trainValidateData.Schema); 


iris-train.tsv found here: c:\Users\xiaoyuz\source\repos\csharp-notebooks\machine-learning\data\iris-train.tsv
iris-test.tsv found here: c:\Users\xiaoyuz\source\repos\csharp-notebooks\machine-learning\data\iris-test.tsv


[ Label: String, Sepal length: Single, Sepal width: Single, Petal length: Single, Petal width: Single ] Count 5 (values) index value 0 Label: String Name Label Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 1 Sepal length: Single Name Sepal length Index 1 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 2 Sepal width: Single Name Sepal width Index 2 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 3 Petal length: Single Name Petal length Index 3 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty) 4 Petal width: Single Name Petal width Index 4 IsHidden False Type Single RawType System.Single Annotations Schema [ ] Count 0 (values) (empty)

In [6]:
display(h4("Showing 5 rows from training DataView:"));
var fewRows = mlContext.Data.CreateEnumerable<ModelInput>(trainValidateData, reuseRowObject: false)
                    .Take(5)
                    .ToList();
display(fewRows);

Showing 5 rows from training DataView:

index value 0 Submission#6+ModelInput Label setosa Sepal_length 5.4 Sepal_width 3.7 Petal_length 1.5 Petal_width 0.2 1 Submission#6+ModelInput Label setosa Sepal_length 4.8 Sepal_width 3.4 Petal_length 1.6 Petal_width 0.2 2 Submission#6+ModelInput Label setosa Sepal_length 4.8 Sepal_width 3 Petal_length 1.4 Petal_width 0.1 3 Submission#6+ModelInput Label setosa Sepal_length 4.3 Sepal_width 3 Petal_length 1.1 Petal_width 0.1 4 Submission#6+ModelInput Label setosa Sepal_length 5.8 Sepal_width 4 Petal_length 1.2 Petal_width 0.2

## Create the training pipeline, choose an algorithm, and train the model:

In [10]:
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.FastTree;
using Microsoft.ML.Trainers;
using Microsoft.ML;

var trainValidateSplit = mlContext.Data.TrainTestSplit(trainValidateData, 0.2);

var trainDataset = trainValidateSplit.TrainSet;
var validateDataset = trainValidateSplit.TestSet;

// Append the trainer to the data processing pipeline
var pipeline = mlContext.Transforms.ReplaceMissingValues(new []{new InputOutputColumnPair(@"Sepal length", @"Sepal length"),new InputOutputColumnPair(@"Sepal width", @"Sepal width"),new InputOutputColumnPair(@"Petal length", @"Petal length"),new InputOutputColumnPair(@"Petal width", @"Petal width")})      
                 .Append(mlContext.Transforms.Concatenate(@"Features", new []{@"Sepal length",@"Sepal width",@"Petal length",@"Petal width"}))      
                 .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName:@"Label",inputColumnName:@"Label"))   
				 .Append(mlContext.Auto().MultiClassification(labelColumnName: "Label"))
                 .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName:@"PredictedLabel",inputColumnName:@"PredictedLabel"));


var monitor = new NotebookMonitor(pipeline);

var experiment = mlContext.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(50)
                    .SetDataset(trainDataset, validateDataset)
                    .SetMulticlassClassificationMetric(MulticlassClassificationMetric.MacroAccuracy, "Label", "PredictedLabel")
                    .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = await experiment.RunAsync();


var model = res.Model;


index,Trial,Metric,Trainer,Parameters
0,0,0.93333334,FastTreeOva=>Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":254,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e8"":{}},""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e2"":{""NumberOfTrees"":4,""NumberOfLeaves"":4,""FeatureFraction"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e3"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":1,""NumberOfTrees"":4,""SubsampleFraction"":1,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e4"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e5"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e6"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e7"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e8"":{}}"
1,1,0.93333334,FastTreeOva=>Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":25,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":213,""FeatureFraction"":0.99999999,""LearningRate"":0.1930972930255831,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e8"":{}},""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e2"":{""NumberOfTrees"":4,""NumberOfLeaves"":4,""FeatureFraction"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e3"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":1,""NumberOfTrees"":4,""SubsampleFraction"":1,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e4"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e5"":{""L1Regularization"":1,""L2Regularization"":1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e6"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e7"":{""L1Regularization"":1,""L2Regularization"":0.1,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features""},""e8"":{}}"
2,2,0.93333334,FastTreeOva=>Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":15,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":302,""FeatureFraction"":0.9384300754394114,""LearningRate"":0.05178736502885681,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e8"":{}},""_SCHEMA_"":""e0 * e1 * e8"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""NumberOfTrees"":4,""MaximumBinCountPerFeature"":255,""FeatureFraction"":1,""LearningRate"":0.09999999999999998,""LabelColumnName"":""Label"",""FeatureColumnName"":""Features"",""DiskTranspose"":false},""e2"":{""NumberOfTrees"":4,""NumberOfLeaves

## Evaluate the model:

In [11]:
// Evaluate the model using the cross validation method
// Learn more about cross validation at https://aka.ms/mlnet-cross-validation


var testDataPredictions = model.Transform(testData);
MulticlassClassificationMetrics trainedModelMetrics = mlContext.MulticlassClassification.Evaluate(testDataPredictions);


trainedModelMetrics

Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 0.19775042201810517 LogLossReduction 0.8199998087971092 MacroAccuracy 0.9666666666666667 MicroAccuracy 0.9666666666666667 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 0.0518951901339455, 0.34421004481758694, 0.1971460311027831 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 1, 0.9090909090909091, 1 ] PerClassRecall [ 1, 1, 0.9 ] Counts index value 0 [ 10, 0, 0 ] 1 [ 0, 10, 0 ] 2 [ 0, 1, 9 ] NumberOfClasses 3

## Consume the model

In [12]:
 // Define sample model input
var sampleData = new ModelInput()
{
    Sepal_length = 4.8F,
    Sepal_width = 3.4F,
    Petal_length = 1.6F,
    Petal_width = 0.2F,
};

// Create a Prediction Engine (used to make single predictions)
var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
// Use the model and Prediction Engine to predict on new sample data
var predictionResult = predEngine.Predict(sampleData);
Console.WriteLine("Using model to make single prediction -- Comparing actual Label with predicted Label from sample data...\n\n");

Console.WriteLine($"Label: {0F}");
Console.WriteLine($"Sepal_length: {4.8F}");
Console.WriteLine($"Sepal_width: {3.4F}");
Console.WriteLine($"Petal_length: {1.6F}");
Console.WriteLine($"Petal_width: {0.2F}");

Console.WriteLine($"\n\nPredicted Label: {predictionResult.PredictedLabel}\n\n");

Using model to make single prediction -- Comparing actual Label with predicted Label from sample data...


Label: 0
Sepal_length: 4.8
Sepal_width: 3.4
Petal_length: 1.6
Petal_width: 0.2


Predicted Label: setosa


